In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

# Chains : secuencias de operaciones
## combina:
    * Un modelo de lenguaje
    * Un prompt
    * Otros componentes

# Simple Chain

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [4]:
chat = ChatOpenAI()

In [6]:
prompt_template = ChatPromptTemplate.from_template(
    "what is the best name to describe a company that makes {product}?"
)
llm_chain = LLMChain(
    llm = chat,
    prompt= prompt_template
)

In [7]:
product = "AI  Applications and AI courses"
llm_chain.run(product)

'"IntelligenTech Solutions"'

## New way to do it with LangChain Expression Language

In [8]:
from langchain.prompts import PromptTemplate
from langchain.schema import StrOutputParser

In [11]:
prompt_template = PromptTemplate.from_template(
    "what is the best name to describe a company that makes {product}?"
)

new_llm = prompt_template | chat | StrOutputParser()
new_llm.invoke(
    {'product':product}
)

'"IntelliTech Solutions"'

# Sequential Chain

In [12]:
from langchain.chains import SequentialChain

In [13]:
# prompt template 1: translate to Spanish
first_prompt = ChatPromptTemplate.from_template(
    "Translate the following text to spanish:"
    "\n\n{Text}"
)

# chain 1: input= Text and output= Spanish_Translation
chain_one = LLMChain(
    llm=chat, 
    prompt=first_prompt, 
    output_key="Spanish_Translation"
)

In [14]:
# prompt template 2: summarize
second_prompt = ChatPromptTemplate.from_template(
    "Can you summarize the following text in 1 sentence:"
    "\n\n{Text}"
)

# chain 2: input= Text and output= summary
chain_two = LLMChain(
    llm=chat, 
    prompt=second_prompt, 
    output_key="summary"
)

In [15]:
# prompt template 3: identify language
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following text:\n\n{Spanish_Translation}"
)

# chain 3: input= Spanish_Translation and output= language
chain_three = LLMChain(
    llm=chat, 
    prompt=third_prompt,
    output_key="language"
)

In [16]:
# prompt template 4: follow up message
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a follow up response to the following "
    "summary in the specified language:"
    "\n\nSummary: {summary}\n\nLanguage: {language}"
)
# chain 4: input= summary, language and output= followup_message
chain_four = LLMChain(
    llm=chat, 
    prompt=fourth_prompt,
    output_key="followup_message"
)

In [21]:
# overall_chain: input= Text 
# and output= Spanish_Translation,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Text"],
    output_variables=["Spanish_Translation","summary","followup_message"],
    verbose=True
)

In [22]:
review = """
Being in a start-up myself, I read this book to find comfort 
and confirmation about the volatility and emotional roller-coaster 
that comes with a working at the brink of software-as-a service. 
This book was just what it promised - interviews from some of the 
great successes in SV and elsewhere, their humble and sometimes 
difficult beginnings ('against all odds') that I am experiencing 
right now. It's not a literary piece of work - never intended to 
be, on the contrary, I felt the writing style was just what fits 
with a life in the fast lane - little time, easy to read chapters, 
inspiring and thank god, very 'down to earth.'

The one critical point I would like to make: I am somewhat 
perplexed how the companies were chosen - there are so many 
other companies that could have fit the profile which seem 
much more of a success than some of the companies/products in 
the book (gmail? Comm'on, I guess the author wanted to have 
Google in there, but didn't get an interview so she went with 
gmail?). Other great companies are easy to find - they don't
even need to be in the consumer space. How about Salesforce.com? 
I definitely liked the mix of 'new' and 'experienced' start ups. 

This book was a breeze to read and insightful for us start-up 
enterpreneurs.
"""
overall_chain(review)



> Entering new SequentialChain chain...

> Finished chain.


{'Text': "\nBeing in a start-up myself, I read this book to find comfort \nand confirmation about the volatility and emotional roller-coaster \nthat comes with a working at the brink of software-as-a service. \nThis book was just what it promised - interviews from some of the \ngreat successes in SV and elsewhere, their humble and sometimes \ndifficult beginnings ('against all odds') that I am experiencing \nright now. It's not a literary piece of work - never intended to \nbe, on the contrary, I felt the writing style was just what fits \nwith a life in the fast lane - little time, easy to read chapters, \ninspiring and thank god, very 'down to earth.'\n\nThe one critical point I would like to make: I am somewhat \nperplexed how the companies were chosen - there are so many \nother companies that could have fit the profile which seem \nmuch more of a success than some of the companies/products in \nthe book (gmail? Comm'on, I guess the author wanted to have \nGoogle in there, but didn

**The new way to do it with LangChain Expression Language**

*Option 1: with the simpler syntax we get just the output.*

In [24]:
from langchain.schema import StrOutputParser

In [25]:
new_sequence_chain = {"Spanish_Translation": first_prompt | chat | StrOutputParser()} | third_prompt | chat | StrOutputParser()

In [26]:
new_sequence_chain.invoke({"Text": review})

'Español.'

*Option 2: with RunnablePassthrough we get input and output.*

In [27]:
from langchain.schema.runnable import RunnablePassthrough

In [34]:
first_chain = first_prompt | chat | StrOutputParser()
second_chain = second_prompt | chat | StrOutputParser()
third_chain = third_prompt | chat | StrOutputParser()
fourth_chain = fourth_prompt | chat | StrOutputParser()

first_plus_third_chain = {"Spanish_Translation": first_chain} | {'summary': second_chain} | RunnablePassthrough.assign(language=third_chain) | RunnablePassthrough.assign(followup_message=fourth_chain)
first_plus_third_chain.invoke({'Text': review})

{'Spanish_Translation': "Estando en una start-up yo mismo, leí este libro para encontrar consuelo y confirmación sobre la volatilidad y montaña rusa emocional que viene con trabajar al borde del software como servicio. Este libro fue justo lo que prometía: entrevistas de algunos de los grandes éxitos en SV y en otros lugares, sus humildes y a veces difíciles comienzos ('contra todo pronóstico') que estoy experimentando en este momento. No es una obra literaria, nunca pretendió serlo, al contrario, sentí que el estilo de escritura era justo lo que encaja con una vida en la rápida carrera: poco tiempo, capítulos fáciles de leer, inspiradores y gracias a Dios, muy 'realistas'.\n\nEl único punto crítico que me gustaría hacer es que estoy algo perplejo sobre cómo se eligieron las empresas: hay tantas otras empresas que podrían haber encajado en el perfil y parecen ser mucho más exitosas que algunas de las empresas/productos en el libro (¿gmail? Vamos, supongo que la autora quería tener a Go